**1: Importing Libraries**

In [1]:
import pandas as pd
import io

**2: Loading the Dataset**

In [2]:
df = pd.read_csv('/content/covid_worldwide.csv')

**3: Converting Numerical Columns to Numeric Type**

In [3]:
numerical_cols = ['Total Cases', 'Total Deaths', 'Total Recovered', 'Active Cases', 'Total Test', 'Population']
for col in numerical_cols:

    df[col] = df[col].replace({',': ''}, regex=True)
    df[col] = pd.to_numeric(df[col], errors='coerce')

Explanation:

*   This cell cleans numerical columns to ensure they are in a proper numeric format (float or integer) for calculations.



------

**4: Dropping Rows with Missing Critical Data**

In [4]:
df = df.dropna(subset=['Country', 'Total Cases'])

Explanation:

*  This cell removes rows where the Country or Total Cases columns have missing values (NaN).
*   Reason: These columns are likely critical for analysis (e.g., Country identifies the data point, and Total Cases is a key metric). Rows missing these values are incomplete and unreliable.

*   The dropna() function with subset ensures only rows with NaN in these specific columns are dropped, preserving as much data as possible.

*   The result is a DataFrame with no missing values in Country or Total Cases.





-----

**5: Imputing Missing Values for Total Recovered or Active Cases**

In [5]:
for idx, row in df.iterrows():
    if pd.isna(row['Total Recovered']) or pd.isna(row['Active Cases']):
        if not pd.isna(row['Total Cases']) and not pd.isna(row['Total Deaths']):

            if not pd.isna(row['Total Recovered']):
                df.at[idx, 'Active Cases'] = row['Total Cases'] - row['Total Deaths'] - row['Total Recovered']

            elif not pd.isna(row['Active Cases']):
                df.at[idx, 'Total Recovered'] = row['Total Cases'] - row['Total Deaths'] - row['Active Cases']

Explanation:

*   This cell fills missing values in Total Recovered or Active Cases using the relationship:
Total Cases = Total Deaths + Total Recovered + Active Cases.




----------

**6: Creating Population Bins**

In [6]:
df['Population_Bin'] = pd.qcut(df['Population'], q=5, labels=False, duplicates='drop')

Explanation:

*   This cell creates a new column, Population_Bin, by dividing the Population column into 5 quantiles (quintiles) using pd.qcut().

-----

**7: Filling Missing Values with Group**

In [7]:
for col in numerical_cols:
    if col != 'Total Cases':
        df[col] = df.groupby('Population_Bin')[col].transform(lambda x: x.fillna(x.median()))

Explanation:

*  This cell fills missing values in numerical columns (except Total Cases) with the median value of their respective Population_Bin group.

-----

**8: Filling Remaining Missing Values with Overall Medians**

In [8]:
for col in numerical_cols:
    df[col] = df[col].fillna(df[col].median())

Explanation:
* This cell fills any remaining missing values in numerical columns with the overall median of each column.

-----

**9: Dropping the Population Bin Column**

In [9]:
df = df.drop(columns=['Population_Bin'])

Explanation:

* This cell removes the Population_Bin column from the DataFrame, as it was only used temporarily for grouping in Cell 7.

-----

**10: Clipping Negative Values**

In [10]:
for col in numerical_cols:
    df[col] = df[col].clip(lower=0)

Explanation:

* This cell ensures all numerical columns have non-negative values by setting any negative values to 0.

-----

**11: Checking Data Quality**

In [11]:
print(f"Rows after cleaning: {len(df)}")
print("Missing values after cleaning:")
print(df.isna().sum())

Rows after cleaning: 231
Missing values after cleaning:
Serial Number      0
Country            0
Total Cases        0
Total Deaths       0
Total Recovered    0
Active Cases       0
Total Test         0
Population         0
dtype: int64


Explanation:

**This cell evaluates the cleaned dataset by:**
* Printing the number of rows remaining after cleaning (len(df)), which is 231.
* Printing the count of missing values per column using df.isna().sum().

-----

**12: Saving the Cleaned Dataset**

In [15]:
df.to_csv('cleaned_covid_worldwide.csv', index=False)

**13: Creating a Dictionary of Missing Value Masks**

In [13]:
missing_masks = {col: df[col].isna() for col in df.columns}

Explanation:

* This cell creates a dictionary missing_masks where each key is a column name, and each value is a boolean Series indicating where values in that column are missing (True for NaN, False otherwise).

------

**14: Comparing Correlations Before and After Cleaning**

In [14]:
corr_before = df.loc[~missing_masks['Total Cases'], ['Total Cases', 'Population']].corr(method='spearman').iloc[0,1]
corr_after = df[['Total Cases', 'Population']].corr(method='spearman').iloc[0,1]
print(f"Correlation before: {corr_before:.4f}, after: {corr_after:.4f}")

Correlation before: 0.6777, after: 0.6777


Explanation:

* This cell calculates and compares the Spearman correlation between Total Cases and Population before and after cleaning.

-----

**15: Checking Data Consistency**

In [21]:
df['Discrepancy'] = df['Total Cases'] - (df['Total Deaths'] + df['Total Recovered'] + df['Active Cases'])
tolerance = 0.01 * df['Total Cases']  # 1% tolerance
consistent = (abs(df['Discrepancy']) <= tolerance).mean() * 100
print(f"\nPercentage of rows where Total Cases = Total Deaths + Total Recovered + Active Cases (within 1%): {consistent:.2f}%")


Percentage of rows where Total Cases = Total Deaths + Total Recovered + Active Cases (within 1%): 87.88%


Explanation:

* This cell checks the consistency of the data by verifying if Total Cases equals the sum of Total Deaths, Total Recovered, and Active Cases within a 1% tolerance.

# Summary:

**The notebook systematically cleans and validates a COVID-19 dataset:**

* **Data Loading and Cleaning:** Loads the dataset, converts numerical columns, removes invalid rows, and imputes missing values using logical relationships and statistical methods (medians).
* **Data Integrity:** Creates population bins for contextual imputation, clips negative values, and verifies no missing values remain.
* **Validation:** Checks data consistency (correlation and discrepancy analysis) to ensure cleaning did not distort key relationships.
* **Output:** Saves the cleaned dataset and provides summary statistics to confirm the process was successful.